## Data curation continued
--- 
`NEW CONTINUING` script from the first [data_curation](../notebooks/data_curation.ipynb) script. 

Data processing pipeline: 
- [`data_curation.ipynb`](../notebooks/data_curation.ipynb)
- `data_curation_cont.ipynb` <<

__Script header__

In [1]:
# loading required libraries
import nltk, pickle, pprint, csv, pylangacq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pretty printing for readability
cp = pprint.PrettyPrinter(compact=True, sort_dicts=True)

# loading data from last notebook


---
### Overview

Continuing my data curation efforts after first [progres report](../progress_report.md). In the last notebook, some issues were encountered, namely, the learner corpus may not be appropriate/contain sufficient data for my analysis. Thus, my first step is to delve into more exploration of that data.

--- 